In [1]:
import pickle
import numpy as np
import pandas as pd
from rectools.dataset import Interactions, Dataset
from rectools.model_selection import TimeRangeSplitter
from rectools.metrics import MAP, MeanInvUserFreq, calc_metrics

from implicit.als import AlternatingLeastSquares

from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools import Columns
from rectools.dataset import Dataset
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel
from tqdm import tqdm

from lightfm import LightFM

In [2]:
interactions_df = pd.read_csv('../artifacts/data_original/interactions.csv')
users = pd.read_csv('../artifacts/data_original/users.csv')
items = pd.read_csv('../artifacts/data_original/items.csv')

Columns.Datetime = 'last_watch_dt'
Columns.Weight = 'watched_pct'
user_ids_all = interactions_df.groupby(Columns.User)[Columns.Item].nunique().reset_index(name='unique_items_count')
hot_users = user_ids_all[user_ids_all['unique_items_count'] > 1][Columns.User]
interactions_df_hot_users = interactions_df[interactions_df[Columns.User].isin(hot_users)]


users = users[users[Columns.User].isin(interactions_df_hot_users[Columns.User])]
interactions_df_hot_users = interactions_df_hot_users[interactions_df_hot_users[Columns.User].isin(users[Columns.User])]
items = items[items[Columns.Item].isin(interactions_df_hot_users[Columns.Item])]

interactions = interactions_df_hot_users   
catalog = interactions[Columns.Item].unique()
interactions

,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
...,...,...,...,...,...
5476244,438585,7829,2021-08-02,6804,100.0
5476245,786732,4880,2021-05-12,753,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476249,384202,16197,2021-04-19,6203,100.0


In [3]:
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')
max_date = interactions[Columns.Datetime].max()

train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train.drop(train.query("total_dur < 300").index, inplace=True)
cold_users = set(test[Columns.User]) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (3806591, 5)
test: (360553, 5)
train: (2938674, 5)
test: (268625, 5)


In [5]:
interactions = Interactions(interactions)

In [6]:
N_SPLITS = 1
TEST_SIZE = '14D'

cv = TimeRangeSplitter(
    test_size=TEST_SIZE,
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=False,
)
cv.get_test_fold_borders(interactions)

[(Timestamp('2021-08-09 00:00:00', freq='14D'),
  Timestamp('2021-08-23 00:00:00', freq='14D'))]

In [7]:
K_RECOS = 10
RANDOM_STATE = 42
NUM_THREADS = 16
N_FACTORS = (32,)
N_EPOCHS = 1 # Lightfm
USER_ALPHA = 0 # Lightfm
ITEM_ALPHA = 0 # Lightfm
LEARNING_RATE = 0.005 # Lightfm

metrics = {
    'map@10': MAP(k=10),
    'novelty': MeanInvUserFreq(k=10)
}

models = {}

implicit_models = {
    'ALS': AlternatingLeastSquares,
}
for implicit_name, implicit_model in implicit_models.items():
    for is_fitting_features in (True, False):
        for n_factors in N_FACTORS:
            models[f"{implicit_name}_{n_factors}_{is_fitting_features}"] = (
                ImplicitALSWrapperModel(
                    model=implicit_model(
                        factors=n_factors, 
                        random_state=RANDOM_STATE, 
                        num_threads=NUM_THREADS,
                    ),
                    fit_features_together=is_fitting_features,
                )
            )

lightfm_losses = ('logistic', 'bpr', 'warp')

for loss in lightfm_losses:
    for n_factors in N_FACTORS:
        models[f"LightFM_{loss}_{n_factors}"] = LightFMWrapperModel(
            LightFM(
                no_components=n_factors, 
                loss=loss, 
                random_state=RANDOM_STATE,
                learning_rate=LEARNING_RATE,
                user_alpha=USER_ALPHA,
                item_alpha=ITEM_ALPHA,
            ),
            epochs=N_EPOCHS,
            num_threads=NUM_THREADS,
        )

/home/vorkov/Workspace/RecoServiceTemplate/venv/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 16 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [8]:
user_features_frames = []
for feature in ["sex", "age", "income"]:
    feature_frame = users.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,М,sex
1,962099,М,sex
3,721985,Ж,sex
4,704055,Ж,sex
5,1037719,М,sex


In [9]:
users

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
5,1037719,age_45_54,income_60_90,М,0
...,...,...,...,...,...
840179,848247,age_18_24,income_20_40,Ж,0
840182,374327,age_35_44,income_20_40,М,1
840188,312839,age_65_inf,income_60_90,Ж,0
840190,393868,age_25_34,income_20_40,М,0


In [10]:
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"

item_features = pd.concat((genre_feature, content_feature))

In [11]:
%%time
dataset = Dataset.construct(
    interactions_df=interactions_df_hot_users,
    user_features_df=user_features,
    cat_user_features=["sex", "age", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

CPU times: user 525 ms, sys: 23.9 ms, total: 549 ms
Wall time: 550 ms


In [12]:
%%time
    
cv.get_test_fold_borders(interactions)

results = []


TEST_USERS = test[Columns.User].unique()


# df_train = interactions.df.iloc[train_ids]
# df_test = interactions.df.iloc[test_ids][Columns.UserItem]
# test_users = np.unique(df_test[Columns.User])

for model_name, model in models.items():
    print(f"Fitting model {model_name}...")
    model_quality = {'model': model_name}

    model.fit(dataset)
    recos = model.recommend(
        users=TEST_USERS,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
    )
    print(recos)
    metric_values = calc_metrics(
            metrics,
            reco=recos,
            interactions=test,
            prev_interactions=interactions.df,
            catalog=catalog,
        )
    model_quality.update(metric_values)
    results.append(model_quality)


Fitting model ALS_32_True...


/home/vorkov/Workspace/RecoServiceTemplate/venv/lib/python3.10/site-packages/rectools/dataset/features.py:420: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")


        user_id  item_id  score  rank
0        200197     8437    NaN     1
1        200197    12173    NaN     2
2        200197     7571    NaN     3
3        200197     1449    NaN     4
4        200197      693    NaN     5
...         ...      ...    ...   ...
920345   442859      354    NaN     6
920346   442859     7638    NaN     7
920347   442859     7107    NaN     8
920348   442859     1659    NaN     9
920349   442859     9506    NaN    10

[920350 rows x 4 columns]
Fitting model ALS_32_False...


ModelFitError: NaN encountered in factors

In [16]:
TEST_USERS

array([200197,  73446,  10010, ..., 436503, 623792, 442859])

In [13]:
df_quality = pd.DataFrame(results).T
df_quality.columns = df_quality.iloc[0]
df_quality.drop('model', inplace=True)

In [14]:
df_quality.style.highlight_max(color='lightgreen', axis=1).data

model,ALS_32_True
map@10,0.003796
novelty,7.773237
